In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf 

from src.pre_processing import *
from src.TechnicalAnalysis import * 
from src.plotting import *

pd.set_option('display.max_columns', 500)

In [14]:
df = pd.read_csv(r'data/trading_activity_03_07_25.csv')
dividends_df = df[df['Action'] == 'Dividend (Dividend)']

df.columns

Index(['Action', 'Time', 'ISIN', 'Ticker', 'Name', 'Notes', 'ID',
       'No. of shares', 'Price / share', 'Currency (Price / share)',
       'Exchange rate', 'Result', 'Currency (Result)', 'Total',
       'Currency (Total)', 'Withholding tax', 'Currency (Withholding tax)',
       'Stamp duty', 'Currency (Stamp duty)', 'Stamp duty reserve tax',
       'Currency (Stamp duty reserve tax)', 'Currency conversion fee',
       'Currency (Currency conversion fee)'],
      dtype='object')

In [ ]:
remove_action_types = ['Dividend adjustment' ,
            'Deposit',
            'Withdrawal',
            'Interest on cash',
            'Dividend adjustment',
            'Stock split open',
            'Stock split close',
            'Dividend (Dividend)'
            ]
df = history_preprocessing(df, remove_action_types)

df['Action'].value_counts()

Action
Market buy     158
Market sell     51
Limit buy       32
Limit sell      22
Name: count, dtype: int64

### Get stock data from yahoo API

In [5]:
yahoo_symbol = 'TSLA'
t212_symbol = 'TSLA'
period = '1y'              
granularity = '1d'

try:
    stock_data = yf.Ticker(yahoo_symbol)
    price_data = stock_data.history(period=period, interval=granularity ) 
except Exception as e:
    logging.error(f"Failed to fetch price data for {yahoo_symbol}: {e}") 

### MACD Calculations  ###

In [6]:
ti = TechnicalIndicators(price_data)
macd = ti.calculate_macd_fast_line(short_span= 12, long_span= 26)
slow_signal = ti.calculate_macd_ema_slow_signal()

### Plot TSLA with custom MACD  ###

In [17]:
plot_stock_price(price_data,
                ticker_name = t212_symbol ,
                candlestick= True,
                # buy_sell_data = df[df['Ticker'] == t212_symbol],  
                ema_periods = [13 ,26],
                plot_macd = True,
                fast_macd = macd,
                slow_signal = slow_signal,
            )